In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from lightgbm import LGBMRegressor

os.chdir('/kaggle/input/g-research-crypto-forecasting')
os.listdir()

In [ ]:
training_df = pd.read_csv('train.csv')
assets_details_data_df = pd.read_csv('asset_details.csv')

In [ ]:
training_df.head()

In [ ]:
assets_details_data_df

In [ ]:
def upper_bound(data):
    return data['High'] - np.maximum(data['Close'], data['Open'])

In [ ]:
def lower_bound(data):
    return np.minimum(data['Close'], data['Open']) - data['Low']

In [ ]:
## Building features from the original data Frame
def get_feature_df(data):
    features_df = data[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()
    features_df['Upper_Shadow'] = upper_bound(features_df)
    features_df['Lower_Shadow'] = lower_bound(features_df)
    return features_df

def get_x_Y_and_assets(df_train, asset_id):
    df =df_train[df_train['Asset_ID'] == asset_id]
    
    new_df = get_feature_df(df)
    new_df['y'] = df['Target']
    new_df = new_df.dropna(how="any")
    
    X = new_df.drop("y", axis=1)
    y = new_df["y"]
    
    model = LGBMRegressor(n_estimators=800)
    model.fit(X, y)
    return X, y, model

In [ ]:
### Iterating over assets
X_s = {}
y_s = {}
models = {}
for asset_id, asset_name in zip(assets_details_data_df['Asset_ID'], assets_details_data_df['Asset_Name']):
    print(f"Training model for {asset_name:<18} (ID={asset_id:<2})")
    X, y, model = get_x_Y_and_assets(training_df, asset_id)
    X_s[asset_id], y_s[asset_id], models[asset_id] = X, y, model

In [ ]:
# Checking the model
x = get_feature_df(training_df.iloc[1])
y_pred = models[0].predict([x])
y_pred[0]

In [ ]:
# os.chdir('/kaggle/input/g-research-crypto-forecasting/gresearch_crypto')
# os.listdir()
##import pdb; pdb.set_trace()

In [ ]:
import gresearch_crypto
#env = gresearch_crypto.make_env()
#iter_test = env.iter_test()

for i, (test_df, pred_df) in enumerate(iter_test):
    for j , row in test_df.iterrows():
        
        model = models[row['Asset_ID']]
        x_test = get_feature_df(row)
        y_pred = model.predict([x_test])[0]
        
        pred_df.loc[pred_df['row_id'] == row['row_id'], 'Target'] = y_pred
        
        if i == 0 and j == 0:
            display(x_test)
    if i == 0:
        display(pred_df)

    # Send submissions
    env.predict('/kaggle/working/pred_df')

In [ ]:
#pred_df
#.to_csv("submission.csv")
#example_df = pd.read_csv('example_sample_submission.csv')
#example_df
pred_df.to_csv('/kaggle/working/submission.csv')